In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 📈 02 - Statistical Analysis: Airlines Dataset\n",
    "## Análise Estatística Inferencial - Voos Delhi-Mumbai\n",
    "\n",
    "**Objetivo**: Realizar testes estatísticos, análises de correlação e inferências sobre os padrões identificados na exploração inicial.\n",
    "\n",
    "**Autor**: [Seu Nome]  \n",
    "**Data**: $(date +\"%Y-%m-%d\")  \n",
    "**Versão**: 1.0\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔧 Setup e Importações"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Importações principais\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import plotly.express as px\n",
    "import plotly.graph_objects as go\n",
    "from plotly.subplots import make_subplots\n",
    "\n",
    "# Importações estatísticas\n",
    "from scipy import stats\n",
    "from scipy.stats import (\n",
    "    normaltest, shapiro, levene, bartlett, \n",
    "    ttest_ind, mannwhitneyu, f_oneway, kruskal,\n",
    "    chi2_contingency, pearsonr, spearmanr,\n",
    "    jarque_bera, anderson\n",
    ")\n",
    "from statsmodels.stats.multicomp import pairwise_tukeyhsd\n",
    "from statsmodels.stats.diagnostic import het_breuschpagan\n",
    "from statsmodels.formula.api import ols\n",
    "import statsmodels.api as sm\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "from sklearn.cluster import KMeans\n",
    "from sklearn.metrics import silhouette_score\n",
    "\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Configurações\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "pd.set_option('display.max_columns', None)\n",
    "\n",
    "print(\"📦 Todas as bibliotecas estatísticas carregadas com sucesso!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📥 Carregamento dos Dados"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Carregar dados processados do notebook anterior\n",
    "df = pd.read_csv('../data/processed/cleaned_flights_data.csv')\n",
    "\n",
    "print(f\"✅ Dataset carregado: {df.shape[0]} linhas × {df.shape[1]} colunas\")\n",
    "\n",
    "# Carregar resumo da exploração\n",
    "import json\n",
    "with open('../data/processed/exploration_summary.json', 'r') as f:\n",
    "    exploration_summary = json.load(f)\n",
    "    \n",
    "print(f\"📊 Resumo da exploração carregado\")\n",
    "print(f\"   • Preço médio anterior: ₹{exploration_summary['price_stats']['mean']:.0f}\")\n",
    "print(f\"   • Voos diretos: {exploration_summary['direct_flights_pct']:.1f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Testes de Normalidade"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Função para testes de normalidade\n",
    "def test_normality(data, variable_name, alpha=0.05):\n",
    "    \"\"\"\n",
    "    Testa normalidade usando múltiplos testes\n",
    "    \"\"\"\n",
    "    print(f\"🔍 TESTE DE NORMALIDADE - {variable_name.upper()}\")\n",
    "    print(\"=\" * 50)\n",
    "    \n",
    "    # Remover valores nulos\n",
    "    clean_data = data.dropna()\n",
    "    n = len(clean_data)\n",
    "    \n",
    "    results = {}\n",
    "    \n",
    "    # Shapiro-Wilk (melhor para n < 5000)\n",
    "    if n <= 5000:\n",
    "        shapiro_stat, shapiro_p = shapiro(clean_data)\n",
    "        results['Shapiro-Wilk'] = {\n",
    "            'statistic': shapiro_stat,\n",
    "            'p_value': shapiro_p,\n",
    "            'is_normal': shapiro_p > alpha\n",
    "        }\n",
    "    \n",
    "    # D'Agostino-Pearson (bom para amostras grandes)\n",
    "    dagostino_stat, dagostino_p = normaltest(clean_data)\n",
    "    results['D\\'Agostino-Pearson'] = {\n",
    "        'statistic': dagostino_stat,\n",
    "        'p_value': dagostino_p,\n",
    "        'is_normal': dagostino_p > alpha\n",
    "    }\n",
    "    \n",
    "    # Jarque-Bera\n",
    "    jb_stat, jb_p = jarque_bera(clean_data)\n",
    "    results['Jarque-Bera'] = {\n",
    "        'statistic': jb_stat,\n",
    "        'p_value': jb_p,\n",
    "        'is_normal': jb_p > alpha\n",
    "    }\n",
    "    \n",
    "    # Anderson-Darling\n",
    "    ad_result = anderson(clean_data, dist='norm')\n",
    "    # Para α = 0.05, usar índice 2 (5%)\n",
    "    ad_critical = ad_result.critical_values[2]  # 5% significance level\n",
    "    ad_is_normal = ad_result.statistic < ad_critical\n",
    "    results['Anderson-Darling'] = {\n",
    "        'statistic': ad_result.statistic,\n",
    "        'critical_value': ad_critical,\n",
    "        'is_normal': ad_is_normal\n",
    "    }\n",
    "    \n",
    "    # Mostrar resultados\n",
    "    for test_name, result in results.items():\n",
    "        if test_name == 'Anderson-Darling':\n",
    "            print(f\"{test_name}:\")\n",
    "            print(f\"  Estatística: {result['statistic']:.4f}\")\n",
    "            print(f\"  Valor Crítico (5%): {result['critical_value']:.4f}\")\n",
    "            print(f\"  Normal? {'✅ Sim' if result['is_normal'] else '❌ Não'}\")\n",
    "        else:\n",
    "            print(f\"{test_name}:\")\n",
    "            print(f\"  Estatística: {result['statistic']:.4f}\")\n",
    "            print(f\"  p-valor: {result['p_value']:.6f}\")\n",
    "            print(f\"  Normal? {'✅ Sim' if result['is_normal'] else '❌ Não'}\")\n",
    "        print()\n",
    "    \n",
    "    # Conclusão\n",
    "    normal_tests = sum([r['is_normal'] for r in results.values()])\n",
    "    total_tests = len(results)\n",
    "    \n",
    "    if normal_tests == total_tests:\n",
    "        conclusion = \"✅ NORMAL - Todos os testes indicam normalidade\"\n",
    "    elif normal_tests == 0:\n",
    "        conclusion = \"❌ NÃO NORMAL - Nenhum teste indica normalidade\"\n",
    "    else:\n",
    "        conclusion = f\"⚠️ INCONCLUSIVO - {normal_tests}/{total_tests} testes indicam normalidade\"\n",
    "    \n",
    "    print(f\"🎯 CONCLUSÃO: {conclusion}\")\n",
    "    print(\"\\n\" + \"=\"*60 + \"\\n\")\n",
    "    \n",
    "    return results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Testar normalidade das variáveis principais\n",
    "variables_to_test = {\n",
    "    'price': df['price'],\n",
    "    'duration': df['duration'],\n",
    "    'days_left': df['days_left']\n",
    "}\n",
    "\n",
    "normality_results = {}\n",
    "for var_name, var_data in variables_to_test.items():\n",
    "    normality_results[var_name] = test_normality(var_data, var_name)\n",
    "\n",
    "# Visualização da normalidade\n",
    "fig, axes = plt.subplots(3, 2, figsize=(15, 15))\n",
    "fig.suptitle('Análise de Normalidade - Q-Q Plots e Histogramas', fontsize=16, fontweight='bold')\n",
    "\n",
    "for i, (var_name, var_data) in enumerate(variables_to_test.items()):\n",
    "    # Q-Q plot\n",
    "    stats.probplot(var_data, dist=\"norm\", plot=axes[i, 0])\n",
    "    axes[i, 0].set_title(f'Q-Q Plot - {var_name.title()}')\n",
    "    axes[i, 0].grid(True)\n",
    "    \n",
    "    # Histograma com curva normal\n",
    "    axes[i, 1].hist(var_data, bins=30, density=True, alpha=0.7, color=f'C{i}')\n",
    "    \n",
    "    # Curva normal teórica\n",
    "    mu, sigma = var_data.mean(), var_data.std()\n",
    "    x = np.linspace(var_data.min(), var_data.max(), 100)\n",
    "    axes[i, 1].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal Teórica')\n",
    "    axes[i, 1].set_title(f'Histograma vs Normal - {var_name.title()}')\n",
    "    axes[i, 1].legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔗 Análise de Correlações"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Matriz de correlação completa\n",
    "print(\"🔗 ANÁLISE DE CORRELAÇÕES\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "# Selecionar apenas variáveis numéricas\n",
    "numeric_cols = ['duration', 'days_left', 'price']\n",
    "correlation_matrix = df[numeric_cols].corr()\n",
    "\n",
    "print(\"📊 Matriz de Correlação de Pearson:\")\n",
    "print(correlation_matrix.round(3))\n",
    "\n",
    "# Teste de significância das correlações\n",
    "def correlation_significance(df, col1, col2, alpha=0.05):\n",
    "    \"\"\"\n",
    "    Testa significância da correlação entre duas variáveis\n",
    "    \"\"\"\n",
    "    # Remover NaNs\n",
    "    data1 = df[col1].dropna()\n",
    "    data2 = df[col2].dropna()\n",
    "    \n",
    "    # Garantir que ambos os arrays tenham o mesmo tamanho\n",
    "    common_idx = df[[col1, col2]].dropna().index\n",
    "    data1 = df.loc[common_idx, col1]\n",
    "    data2 = df.loc[common_idx, col2]\n",
    "    \n",
    "    # Pearson\n",
    "    pearson_r, pearson_p = pearsonr(data1, data2)\n",
    "    \n",
    "    # Spearman (não-paramétrico)\n",
    "    spearman_r, spearman_p = spearmanr(data1, data2)\n",
    "    \n",
    "    return {\n",
    "        'pearson': {'r': pearson_r, 'p': pearson_p, 'significant': pearson_p < alpha},\n",
    "        'spearman': {'r': spearman_r, 'p': spearman_p, 'significant': spearman_p < alpha}\n",
    "    }\n",
    "\n",
    "# Testar todas as combinações de correlações\n",
    "correlation_tests = {}\n",
    "for i, col1 in enumerate(numeric_cols):\n",
    "    for j, col2 in enumerate(numeric_cols):\n",
    "        if i < j:  # Evitar duplicatas\n",
    "            pair = f\"{col1}_vs_{col2}\"\n",
    "            correlation_tests[pair] = correlation_significance(df, col1, col2)\n",
    "\n",
    "print(\"\\n🎯 TESTES DE SIGNIFICÂNCIA DAS CORRELAÇÕES\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "for pair, results in correlation_tests.items():\n",
    "    col1, col2 = pair.split('_vs_')\n",
    "    print(f\"\\n{col1.upper()} vs {col2.upper()}:\")\n",
    "    print(f\"  Pearson: r = {results['pearson']['r']:.4f}, p = {results['pearson']['p']:.6f} {'✅' if results['pearson']['significant'] else '❌'}\")\n",
    "    print(f\"  Spearman: ρ = {results['spearman']['r']:.4f}, p = {results['spearman']['p']:.6f} {'✅' if results['spearman']['significant'] else '❌'}\")\n",
    "    \n",
    "    # Interpretação da força da correlação\n",
    "    r = abs(results['pearson']['r'])\n",
    "    if r < 0.1:\n",
    "        strength = \"negligível\"\n",
    "    elif r < 0.3:\n",
    "        strength = \"fraca\"\n",
    "    elif r < 0.5:\n",
    "        strength = \"moderada\"\n",
    "    elif r < 0.7:\n",
    "        strength = \"forte\"\n",
    "    else:\n",
    "        strength = \"muito forte\"\n",
    "    \n",
    "    print(f\"  Interpretação: Correlação {strength}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualização da matriz de correlação\n",
    "plt.figure(figsize=(10, 8))\n",
    "\n",
    "# Heatmap com anotações\n",
    "mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))\n",
    "sns.heatmap(correlation_matrix, \n",
    "            mask=mask,\n",
    "            annot=True, \n",
    "            cmap='RdBu_r', \n",
    "            center=0,\n",
    "            square=True,\n",
    "            fmt='.3f',\n",
    "            cbar_kws={\"shrink\": .8})\n",
    "\n",
    "plt.title('Matriz de Correlação - Variáveis Numéricas', fontsize=14, fontweight='bold')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Scatter plots das correlações principais\n",
    "fig = make_subplots(\n",
    "    rows=1, cols=2,\n",
    "    subplot_titles=('Preço vs Duração', 'Preço vs Dias Restantes')\n",
    ")\n",
    "\n",
    "# Scatter plot preço vs duração\n",
    "fig.add_trace(\n",
    "    go.Scatter(x=df['duration'], y=df['price'], \n",
    "               mode='markers', \n",
    "               name='Preço vs Duração',\n",
    "               text=df['airline'],\n",
    "               marker=dict(size=8, opacity=0.6)),\n",
    "    row=1, col=1\n",
    ")\n",
    "\n",
    "# Scatter plot preço vs dias_left\n",
    "fig.add_trace(\n",
    "    go.Scatter(x=df['days_left'], y=df['price'], \n",
    "               mode='markers', \n",
    "               name='Preço vs Dias',\n",
    "               text=df['airline'],\n",
    "               marker=dict(size=8, opacity=0.6)),\n",
    "    row=1, col=2\n",
    ")\n",
    "\n",
    "fig.update_layout(height=500, title_text=\"Análise de Correlações - Scatter Plots\")\n",
    "fig.update_xaxes(title_text=\"Duração (horas)\", row=1, col=1)\n",
    "fig.update_xaxes(title_text=\"Dias Restantes\", row=1, col=2)\n",
    "fig.update_yaxes(title_text=\"Preço (₹)\", row=1, col=1)\n",
    "fig.update_yaxes(title_text=\"Preço (₹)\", row=1, col=2)\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✈️ Comparação Entre Companhias Aéreas"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ANOVA - Teste se há diferença significativa entre médias de preços das airlines\n",
    "print(\"✈️ ANÁLISE ESTATÍSTICA - PREÇOS POR COMPANHIA AÉREA\")\n",
    "print(\"=\" * 60)\n",
    "\n",
    "# Preparar dados por airline\n",
    "airline_groups = [df[df['airline'] == airline]['price'].values for airline in df['airline'].unique()]\n",
    "airline_names = df['airline'].unique()\n",
    "\n",
    "# Teste de homogeneidade das variâncias (Levene)\n",
    "levene_stat, levene_p = levene(*airline_groups)\n",
    "print(f\"🔍 TESTE DE LEVENE (Homogeneidade das Variâncias):\")\n",
    "print(f\"   Estatística: {levene_stat:.4f}\")\n",
    "print(f\"   p-valor: {levene_p:.6f}\")\n",
    "print(f\"   Variâncias homogêneas? {'✅ Sim' if levene_p > 0.05 else '❌ Não'}\")\n",
    "\n",
    "# Bartlett test (mais sensível à normalidade)\n",
    "bartlett_stat, bartlett_p = bartlett(*airline_groups)\n",
    "print(f\"\\n🔍 TESTE DE BARTLETT (Homogeneidade - sensível à normalidade):\")\n",
    "print(f\"   Estatística: {bartlett_stat:.4f}\")\n",
    "print(f\"   p-valor: {bartlett_p:.6f}\")\n",
    "print(f\"   Variâncias homogêneas? {'✅ Sim' if bartlett_p > 0.05 else '❌ Não'}\")\n",
    "\n",
    "# ANOVA ou Kruskal-Wallis dependendo da normalidade e homogeneidade\n",
    "print(f\"\\n📊 TESTE DE DIFERENÇA ENTRE GRUPOS:\")\n",
    "\n",
    "# ANOVA paramétrico\n",
    "f_stat, f_p = f_oneway(*airline_groups)\n",
    "print(f\"   ANOVA F-test:\")\n",
    "print(f\"     F-estatística: {f_stat:.4f}\")\n",
    "print(f\"     p-valor: {f_p:.6f}\")\n",
    "print(f\"     Diferença significativa? {'✅ Sim' if f_p < 0.05 else '❌ Não'}\")\n",
    "\n",
    "# Kruskal-Wallis não-paramétrico\n",
    "kw_stat, kw_p = kruskal(*airline_groups)\n",
    "print(f\"   Kruskal-Wallis:\")\n",
    "print(f\"     H-estatística: {kw_stat:.4f}\")\n",
    "print(f\"     p-valor: {kw_p:.6f}\")\n",
    "print(f\"     Diferença significativa? {'✅ Sim' if kw_p < 0.05 else '❌ Não'}\")\n",
    "\n",
    "# Se houver diferença significativa, fazer teste post-hoc\n",
    "if f_p < 0.05:\n",
    "    print(f\"\\n🎯 TESTE POST-HOC (Tukey HSD):\")\n",
    "    \n",
    "    # Preparar dados para Tukey\n",
    "    tukey_data = []\n",
    "    tukey_labels = []\n",
    "    \n",
    "    for airline in df['airline'].unique():\n",
    "        prices = df[df['airline'] == airline]['price'].values\n",
    "        tukey_data.extend(prices)\n",
    "        tukey_labels.extend([airline] * len(prices))\n",
    "    \n",
    "    tukey_result = pairwise_tukeyhsd(tukey_data, tukey_labels, alpha=0.05)\n",
    "    print(tukey_result)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Estatísticas descritivas por airline\n",
    "print(\"📊 ESTATÍSTICAS DESCRITIVAS POR COMPANHIA AÉREA\")\n",
    "print(\"=\" * 60)\n",
    "\n",
    "airline_stats = df.groupby('airline')['price'].agg([\n",
    "    'count', 'mean', 'std', 'min', 'max', 'median',\n",
    "    lambda x: x.quantile(0.25),  # Q1\n",
    "    lambda x: x.quantile(0.75)   # Q3\n",
    "]).round(0)\n",
    "\n",
    "airline_stats.columns = ['Count', 'Mean', 'Std', 'Min', 'Max', 'Median', 'Q1', 'Q3']\n",
    "airline_stats = airline_stats.sort_values('Mean', ascending=False)\n",
    "\n",
    "# Calcular coeficiente de variação\n",
    "airline_stats['CV'] = (airline_stats['Std'] / airline_stats['Mean'] * 100).round(1)\n",
    "\n",
    "# Calcular IQR\n",
    "airline_stats['IQR'] = airline_stats['Q3'] - airline_stats['Q1']\n",
    "\n",
    "print(airline_stats)\n",
    "\n",
    "print(f\"\\n📈 COEFICIENTE DE VARIAÇÃO (Std/Mean × 100):\")\n",
    "cv_sorted = airline_stats['CV'].sort_values()\n",
    "for airline, cv in cv_sorted.items():\n",
    "    stability = \"estável\" if cv < 20 else \"moderada\" if cv < 40 else \"alta\"\n",
    "    print(f\"   {airline}: {cv}% - Variabilidade {stability}\")\n",
    "\n",
    "# Análise de eficiência (preço por hora de voo)\n",
    "print(f\"\\n⚡ ANÁLISE DE EFICIÊNCIA (Preço por Hora):\")\n",
    "efficiency_stats = df.groupby('airline').apply(lambda x: (x['price'] / x['duration']).mean()).sort_values()\n",
    "for airline, efficiency in efficiency_stats.items():\n",
    "    print(f\"   {airline}: ₹{efficiency:.0f}/hora\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ⏰ Análise Temporal - Horários de Partida"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    # Análise estatística dos preços por horário de partida
print("🕐 ANÁLISE ESTATÍSTICA - PREÇOS POR HORÁRIO DE PARTIDA")
print("=" * 60)

# Preparar dados por horário
time_groups = [df[df['departure_time'] == time]['price'].values for time in df['departure_time'].unique()]
time_names = df['departure_time'].unique()

# Teste de homogeneidade das variâncias
levene_stat_time, levene_p_time = levene(*time_groups)
print(f"🔍 TESTE DE LEVENE (Homogeneidade das Variâncias):")
print(f"   Estatística: {levene_stat_time:.4f}")
print(f"   p-valor: {levene_p_time:.6f}")
print(f"   Variâncias homogêneas? {'✅ Sim' if levene_p_time > 0.05 else '❌ Não'}")

# ANOVA para horários de partida
f_stat_time, f_p_time = f_oneway(*time_groups)
print(f"\n📊 ANOVA - PREÇOS POR HORÁRIO:")
print(f"   F-estatística: {f_stat_time:.4f}")
print(f"   p-valor: {f_p_time:.6f}")
print(f"   Diferença significativa? {'✅ Sim' if f_p_time < 0.05 else '❌ Não'}")

# Kruskal-Wallis (não-paramétrico)
kw_stat_time, kw_p_time = kruskal(*time_groups)
print(f"\n   Kruskal-Wallis:")
print(f"   H-estatística: {kw_stat_time:.4f}")
print(f"   p-valor: {kw_p_time:.6f}")
print(f"   Diferença significativa? {'✅ Sim' if kw_p_time < 0.05 else '❌ Não'}")

# Post-hoc test se significativo
if f_p_time < 0.05:
    print(f"\n🎯 TESTE POST-HOC (Tukey HSD) - HORÁRIOS:")
    
    # Preparar dados para Tukey
    time_data = []
    time_labels = []
    
    for time in df['departure_time'].unique():
        prices = df[df['departure_time'] == time]['price'].values
        time_data.extend(prices)
        time_labels.extend([time] * len(prices))
    
    tukey_time_result = pairwise_tukeyhsd(time_data, time_labels, alpha=0.05)
    print(tukey_time_result)

## 🛑 Análise de Paradas (Stops) - Teste T
print("\n" + "="*60)
print("🛑 ANÁLISE ESTATÍSTICA - VOOS DIRETOS vs COM PARADAS")
print("=" * 60)

# Separar grupos
direct_flights = df[df['stops'] == 'zero']['price'].values
flights_with_stops = df[df['stops'] == 'one']['price'].values

print(f"📊 ESTATÍSTICAS BÁSICAS:")
print(f"   Voos diretos: n={len(direct_flights)}, média=₹{np.mean(direct_flights):.0f}, std=₹{np.std(direct_flights):.0f}")
print(f"   Voos com paradas: n={len(flights_with_stops)}, média=₹{np.mean(flights_with_stops):.0f}, std=₹{np.std(flights_with_stops):.0f}")

# Teste de homogeneidade das variâncias
levene_stat_stops, levene_p_stops = levene(direct_flights, flights_with_stops)
print(f"\n🔍 TESTE DE LEVENE:")
print(f"   Estatística: {levene_stat_stops:.4f}")
print(f"   p-valor: {levene_p_stops:.6f}")
print(f"   Variâncias homogêneas? {'✅ Sim' if levene_p_stops > 0.05 else '❌ Não'}")

# Teste t independente
equal_var = levene_p_stops > 0.05
t_stat, t_p = ttest_ind(direct_flights, flights_with_stops, equal_var=equal_var)
print(f"\n📊 TESTE T INDEPENDENTE:")
print(f"   t-estatística: {t_stat:.4f}")
print(f"   p-valor: {t_p:.6f}")
print(f"   Diferença significativa? {'✅ Sim' if t_p < 0.05 else '❌ Não'}")

# Mann-Whitney U (não-paramétrico)
u_stat, u_p = mannwhitneyu(direct_flights, flights_with_stops, alternative='two-sided')
print(f"\n   Mann-Whitney U:")
print(f"   U-estatística: {u_stat:.4f}")
print(f"   p-valor: {u_p:.6f}")
print(f"   Diferença significativa? {'✅ Sim' if u_p < 0.05 else '❌ Não'}")

# Cálculo do tamanho do efeito (Cohen's d)
def cohens_d(x, y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    pooled_std = np.sqrt(((nx-1)*np.std(x, ddof=1)**2 + (ny-1)*np.std(y, ddof=1)**2) / dof)
    return (np.mean(x) - np.mean(y)) / pooled_std

effect_size = cohens_d(direct_flights, flights_with_stops)
print(f"\n📏 TAMANHO DO EFEITO (Cohen's d): {effect_size:.3f}")
if abs(effect_size) < 0.2:
    effect_interpretation = "pequeno"
elif abs(effect_size) < 0.5:
    effect_interpretation = "médio"
elif abs(effect_size) < 0.8:
    effect_interpretation = "grande"
else:
    effect_interpretation = "muito grande"
print(f"   Interpretação: Efeito {effect_interpretation}")

## 📈 Modelagem de Regressão Linear
print("\n" + "="*60)
print("📈 ANÁLISE DE REGRESSÃO LINEAR")
print("=" * 60)

# Preparar variáveis para regressão
X = df[['duration', 'days_left']].copy()
y = df['price'].copy()

# Adicionar variáveis dummy para categóricas
airline_dummies = pd.get_dummies(df['airline'], prefix='airline')
stops_dummies = pd.get_dummies(df['stops'], prefix='stops')
time_dummies = pd.get_dummies(df['departure_time'], prefix='time')

# Combinar todas as variáveis (removendo uma categoria de cada para evitar multicolinearidade)
X_full = pd.concat([
    X,
    airline_dummies.iloc[:, :-1],  # Remove last airline
    stops_dummies.iloc[:, :-1],    # Remove last stop category
    time_dummies.iloc[:, :-1]      # Remove last time category
], axis=1)

# Adicionar constante
X_full_const = sm.add_constant(X_full)

# Ajustar modelo
model = sm.OLS(y, X_full_const).fit()

print("🔍 RESULTADOS DA REGRESSÃO:")
print(model.summary())

# Testes de diagnóstico do modelo
print(f"\n🔬 DIAGNÓSTICOS DO MODELO:")
print(f"   R² Ajustado: {model.rsquared_adj:.4f}")
print(f"   AIC: {model.aic:.2f}")
print(f"   BIC: {model.bic:.2f}")
print(f"   F-statistic: {model.fvalue:.2f} (p={model.f_pvalue:.6f})")

# Teste de heterocedasticidade (Breusch-Pagan)
bp_stat, bp_p, bp_f_stat, bp_f_p = het_breuschpagan(model.resid, model.model.exog)
print(f"\n🔍 TESTE DE HETEROCEDASTICIDADE (Breusch-Pagan):")
print(f"   Estatística: {bp_stat:.4f}")
print(f"   p-valor: {bp_p:.6f}")
print(f"   Homocedasticidade? {'✅ Sim' if bp_p > 0.05 else '❌ Não'}")

# Análise de resíduos
residuals = model.resid
print(f"\n📊 ANÁLISE DE RESÍDUOS:")
print(f"   Média dos resíduos: {np.mean(residuals):.6f}")
print(f"   Desvio padrão dos resíduos: {np.std(residuals):.2f}")

# Teste de normalidade dos resíduos
shapiro_resid_stat, shapiro_resid_p = shapiro(residuals)
print(f"   Normalidade dos resíduos (Shapiro): p={shapiro_resid_p:.6f} {'✅' if shapiro_resid_p > 0.05 else '❌'}")

# Visualização dos diagnósticos da regressão
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Diagnósticos da Regressão Linear', fontsize=16, fontweight='bold')

# Resíduos vs Valores preditos
axes[0, 0].scatter(model.fittedvalues, residuals, alpha=0.6)
axes[0, 0].axhline(y=0, color='red', linestyle='--')
axes[0, 0].set_xlabel('Valores Preditos')
axes[0, 0].set_ylabel('Resíduos')
axes[0, 0].set_title('Resíduos vs Valores Preditos')

# Q-Q plot dos resíduos
stats.probplot(residuals, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('Q-Q Plot - Resíduos')

# Histograma dos resíduos
axes[1, 0].hist(residuals, bins=30, density=True, alpha=0.7)
mu, sigma = np.mean(residuals), np.std(residuals)
x = np.linspace(residuals.min(), residuals.max(), 100)
axes[1, 0].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2)
axes[1, 0].set_xlabel('Resíduos')
axes[1, 0].set_ylabel('Densidade')
axes[1, 0].set_title('Distribuição dos Resíduos')

# Scale-Location plot
standardized_resid = np.sqrt(np.abs(stats.zscore(residuals)))
axes[1, 1].scatter(model.fittedvalues, standardized_resid, alpha=0.6)
axes[1, 1].set_xlabel('Valores Preditos')
axes[1, 1].set_ylabel('√|Resíduos Padronizados|')
axes[1, 1].set_title('Scale-Location Plot')

plt.tight_layout()
plt.show()

## 🎯 Análise de Clustering
print("\n" + "="*60)
print("🎯 ANÁLISE DE CLUSTERING - SEGMENTAÇÃO DE MERCADO")
print("=" * 60)

# Preparar dados para clustering
clustering_data = df[['price', 'duration', 'days_left']].copy()

# Padronizar os dados
scaler = StandardScaler()
clustering_scaled = scaler.fit_transform(clustering_data)

# Determinar número ótimo de clusters usando método do cotovelo
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(clustering_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(clustering_scaled, kmeans.labels_))

# Visualizar métrica do cotovelo e silhueta
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(K_range, inertias, 'bo-')
axes[0].set_xlabel('Número de Clusters (k)')
axes[0].set_ylabel('Inércia')
axes[0].set_title('Método do Cotovelo')
axes[0].grid(True)

axes[1].plot(K_range, silhouette_scores, 'ro-')
axes[1].set_xlabel('Número de Clusters (k)')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Análise de Silhueta')
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Escolher número ótimo de clusters (maior silhouette score)
optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"🎯 NÚMERO ÓTIMO DE CLUSTERS: {optimal_k}")
print(f"   Silhouette Score: {max(silhouette_scores):.4f}")

# Aplicar clustering com k ótimo
kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans_optimal.fit_predict(clustering_scaled)

# Adicionar labels ao dataframe original
df_clustered = df.copy()
df_clustered['cluster'] = cluster_labels

# Analisar características de cada cluster
print(f"\n📊 ANÁLISE DOS CLUSTERS:")
print("=" * 50)

cluster_summary = df_clustered.groupby('cluster').agg({
    'price': ['count', 'mean', 'std', 'min', 'max'],
    'duration': 'mean',
    'days_left': 'mean'
}).round(2)

cluster_summary.columns = ['Count', 'Price_Mean', 'Price_Std', 'Price_Min', 'Price_Max', 'Duration_Mean', 'DaysLeft_Mean']

print(cluster_summary)

# Análise detalhada de cada cluster
for cluster_id in range(optimal_k):
    cluster_data = df_clustered[df_clustered['cluster'] == cluster_id]
    print(f"\n🏷️ CLUSTER {cluster_id}:")
    print(f"   Tamanho: {len(cluster_data)} voos ({len(cluster_data)/len(df)*100:.1f}%)")
    print(f"   Preço médio: ₹{cluster_data['price'].mean():.0f}")
    print(f"   Duração média: {cluster_data['duration'].mean():.1f} horas")
    print(f"   Airlines principais: {dict(cluster_data['airline'].value_counts().head(3))}")
    print(f"   Voos diretos: {(cluster_data['stops'] == 'zero').mean()*100:.1f}%")
    
    # Caracterizar o cluster
    if cluster_data['price'].mean() < df['price'].quantile(0.33):
        cluster_type = "BUDGET"
    elif cluster_data['price'].mean() > df['price'].quantile(0.67):
        cluster_type = "PREMIUM"
    else:
        cluster_type = "MAINSTREAM"
    
    print(f"   Tipo: {cluster_type}")

# Visualização dos clusters
fig = plt.figure(figsize=(15, 10))

# 3D scatter plot
ax1 = fig.add_subplot(221, projection='3d')
scatter = ax1.scatter(df_clustered['price'], df_clustered['duration'], df_clustered['days_left'], 
                     c=cluster_labels, cmap='viridis', alpha=0.6)
ax1.set_xlabel('Preço')
ax1.set_ylabel('Duração')
ax1.set_zlabel('Dias Restantes')
ax1.set_title('Clusters 3D')
plt.colorbar(scatter, ax=ax1)

# 2D scatter plots
ax2 = fig.add_subplot(222)
scatter2 = ax2.scatter(df_clustered['price'], df_clustered['duration'], 
                      c=cluster_labels, cmap='viridis', alpha=0.6)
ax2.set_xlabel('Preço')
ax2.set_ylabel('Duração')
ax2.set_title('Clusters: Preço vs Duração')

ax3 = fig.add_subplot(223)
scatter3 = ax3.scatter(df_clustered['price'], df_clustered['days_left'], 
                      c=cluster_labels, cmap='viridis', alpha=0.6)
ax3.set_xlabel('Preço')
ax3.set_ylabel('Dias Restantes')
ax3.set_title('Clusters: Preço vs Dias Restantes')

# Distribuição dos clusters por airline
ax4 = fig.add_subplot(224)
cluster_airline = pd.crosstab(df_clustered['airline'], df_clustered['cluster'], normalize='index')
cluster_airline.plot(kind='bar', stacked=True, ax=ax4)
ax4.set_title('Distribuição dos Clusters por Airline')
ax4.set_xlabel('Airline')
ax4.set_ylabel('Proporção')
ax4.legend(title='Cluster')

plt.tight_layout()
plt.show()

# Análise de associação entre variáveis categóricas
print("\n" + "="*60)
print("🔗 ANÁLISE DE ASSOCIAÇÃO - VARIÁVEIS CATEGÓRICAS")
print("=" * 60)

# Teste Chi-quadrado: Airline vs Stops
contingency_airline_stops = pd.crosstab(df['airline'], df['stops'])
chi2_stat, chi2_p, chi2_dof, chi2_expected = chi2_contingency(contingency_airline_stops)

print("✈️ TESTE CHI-QUADRADO: AIRLINE vs STOPS")
print(f"   Estatística Chi²: {chi2_stat:.4f}")
print(f"   p-valor: {chi2_p:.6f}")
print(f"   Graus de liberdade: {chi2_dof}")
print(f"   Associação significativa? {'✅ Sim' if chi2_p < 0.05 else '❌ Não'}")

print(f"\n📊 TABELA DE CONTINGÊNCIA:")
print(contingency_airline_stops)

# Coeficiente de contingência (força da associação)
n = contingency_airline_stops.sum().sum()
contingency_coeff = np.sqrt(chi2_stat / (chi2_stat + n))
print(f"\n📏 COEFICIENTE DE CONTINGÊNCIA: {contingency_coeff:.4f}")

# Teste Chi-quadrado: Departure Time vs Price Range
df['price_category'] = pd.qcut(df['price'], q=3, labels=['Baixo', 'Médio', 'Alto'])
contingency_time_price = pd.crosstab(df['departure_time'], df['price_category'])
chi2_time_stat, chi2_time_p, chi2_time_dof, chi2_time_expected = chi2_contingency(contingency_time_price)

print(f"\n🕐 TESTE CHI-QUADRADO: DEPARTURE_TIME vs PRICE_CATEGORY")
print(f"   Estatística Chi²: {chi2_time_stat:.4f}")
print(f"   p-valor: {chi2_time_p:.6f}")
print(f"   Associação significativa? {'✅ Sim' if chi2_time_p < 0.05 else '❌ Não'}")

# Análise de variância explicada
print("\n" + "="*60)
print("📈 ANÁLISE DE VARIÂNCIA EXPLICADA")
print("=" * 60)

# ANOVA para cada variável categórica
print("🔍 VARIÂNCIA EXPLICADA POR CADA FATOR:")

# Airline
airline_groups = [df[df['airline'] == airline]['price'] for airline in df['airline'].unique()]
f_airline, p_airline = f_oneway(*airline_groups)
eta_squared_airline = (f_airline * (len(df['airline'].unique()) - 1)) / (f_airline * (len(df['airline'].unique()) - 1) + len(df) - len(df['airline'].unique()))

print(f"   Airline: η² = {eta_squared_airline:.4f} ({eta_squared_airline*100:.1f}% da variância)")

# Departure Time  
time_groups = [df[df['departure_time'] == time]['price'] for time in df['departure_time'].unique()]
f_time, p_time = f_oneway(*time_groups)
eta_squared_time = (f_time * (len(df['departure_time'].unique()) - 1)) / (f_time * (len(df['departure_time'].unique()) - 1) + len(df) - len(df['departure_time'].unique()))

print(f"   Departure Time: η² = {eta_squared_time:.4f} ({eta_squared_time*100:.1f}% da variância)")

# Stops
stops_groups = [df[df['stops'] == stop]['price'] for stop in df['stops'].unique()]
f_stops, p_stops = f_oneway(*stops_groups)
eta_squared_stops = (f_stops * (len(df['stops'].unique()) - 1)) / (f_stops * (len(df['stops'].unique()) - 1) + len(df) - len(df['stops'].unique()))

print(f"   Stops: η² = {eta_squared_stops:.4f} ({eta_squared_stops*100:.1f}% da variância)")

# Análise de interações
print("\n🔗 ANÁLISE DE INTERAÇÕES ENTRE FATORES:")

# Interação Airline × Stops
interaction_data = []
for airline in df['airline'].unique():
    for stop in df['stops'].unique():
        subset = df[(df['airline'] == airline) & (df['stops'] == stop)]
        if len(subset) > 0:
            interaction_data.append({
                'airline': airline,
                'stops': stop,
                'mean_price': subset['price'].mean(),
                'count': len(subset)
            })

interaction_df = pd.DataFrame(interaction_data)
interaction_pivot = interaction_df.pivot(index='airline', columns='stops', values='mean_price')

print("💰 PREÇO MÉDIO POR AIRLINE × STOPS:")
print(interaction_pivot.round(0))

# Teste de robustez dos resultados
print("\n" + "="*60)
print("🛡️ TESTES DE ROBUSTEZ")
print("=" * 60)

# Bootstrap para intervalos de confiança das médias
from scipy import stats
import random

def bootstrap_mean(data, n_bootstrap=1000, confidence=0.95):
    """Calcula intervalo de confiança bootstrap para a média"""
    bootstrap_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        bootstrap_means.append(np.mean(sample))
    
    alpha = 1 - confidence
    lower = np.percentile(bootstrap_means, (alpha/2) * 100)
    upper = np.percentile(bootstrap_means, (1 - alpha/2) * 100)
    
    return lower, upper

print("🔄 INTERVALOS DE CONFIANÇA BOOTSTRAP (95%):")
print("   Preços por Airline:")

for airline in df['airline'].unique():
    airline_prices = df[df['airline'] == airline]['price'].values
    if len(airline_prices) > 10:  # Apenas se tiver dados suficientes
        ci_lower, ci_upper = bootstrap_mean(airline_prices)
        mean_price = np.mean(airline_prices)
        print(f"     {airline}: ₹{mean_price:.0f} [₹{ci_lower:.0f}, ₹{ci_upper:.0f}]")

# Análise de sensibilidade - removendo outliers
print(f"\n🎯 ANÁLISE DE SENSIBILIDADE - REMOVENDO OUTLIERS:")

# Identificar outliers usando IQR
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_no_outliers = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
print(f"   Dados originais: {len(df)} observações")
print(f"   Sem outliers: {len(df_no_outliers)} observações")
print(f"   Outliers removidos: {len(df) - len(df_no_outliers)} ({(len(df) - len(df_no_outliers))/len(df)*100:.1f}%)")

# Recomputar testes principais sem outliers
airline_groups_clean = [df_no_outliers[df_no_outliers['airline'] == airline]['price'].values 
                       for airline in df_no_outliers['airline'].unique()]
f_clean, p_clean = f_oneway(*airline_groups_clean)

print(f"\n   ANOVA sem outliers:")
print(f"     F-estatística: {f_clean:.4f} (original: {f_stat:.4f})")
print(f"     p-valor: {p_clean:.6f} (original: {f_p:.6f})")
print(f"     Resultado {'mantido' if (p_clean < 0.05) == (f_p < 0.05) else 'alterado'}")

# Resumo executivo dos testes estatísticos
print("\n" + "="*60)
print("📋 RESUMO EXECUTIVO - TESTES ESTATÍSTICOS")
print("=" * 60)

summary_results = {
    "Normalidade dos Dados": {
        "price": "Não normal (p < 0.001)" if normality_results['price']['Shapiro-Wilk']['p_value'] < 0.001 else "Normal",
        "duration": "Não normal (p < 0.001)" if normality_results['duration']['Shapiro-Wilk']['p_value'] < 0.001 else "Normal",
        "implicacao": "Usar testes não-paramétricos quando apropriado"
    },
    "Diferenças entre Airlines": {
        "ANOVA": f"F={f_stat:.2f}, p={f_p:.6f}",
        "significativo": "Sim" if f_p < 0.05 else "Não",
        "interpretacao": "Diferenças significativas entre airlines" if f_p < 0.05 else "Sem diferenças significativas"
    },
    "Diferenças entre Horários": {
        "ANOVA": f"F={f_stat_time:.2f}, p={f_p_time:.6f}",
        "significativo": "Sim" if f_p_time < 0.05 else "Não",
        "interpretacao": "Diferenças significativas entre horários" if f_p_time < 0.05 else "Sem diferenças significativas"
    },
    "Voos Diretos vs Paradas": {
        "teste_t": f"t={t_stat:.2f}, p={t_p:.6f}",
        "significativo": "Sim" if t_p < 0.05 else "Não",
        "tamanho_efeito": f"Cohen's d = {effect_size:.3f} (efeito {effect_interpretation})",
        "interpretacao": "Voos diretos significativamente diferentes" if t_p < 0.05 else "Sem diferença significativa"
    },
    "Modelo de Regressão": {
        "R_quadrado": f"{model.rsquared:.4f}",
        "R_quadrado_ajustado": f"{model.rsquared_adj:.4f}",
        "interpretacao": f"Modelo explica {model.rsquared_adj*100:.1f}% da variância nos preços"
    },
    "Segmentação (Clustering)": {
        "clusters_otimos": optimal_k,
        "silhouette_score": f"{max(silhouette_scores):.4f}",
        "interpretacao": "Segmentação clara do mercado identificada"
    }
}

for category, results in summary_results.items():
    print(f"\n🎯 {category}:")
    for key, value in results.items():
        if key != "interpretacao":
            print(f"   {key}: {value}")
    print(f"   → {results.get('interpretacao', '')}")

# Exportar resultados da análise estatística
statistical_results = {
    "normality_tests": normality_results,
    "correlation_analysis": correlation_tests,
    "anova_results": {
        "airlines": {"F": f_stat, "p": f_p, "significant": f_p < 0.05},
        "departure_times": {"F": f_stat_time, "p": f_p_time, "significant": f_p_time < 0.05},
        "stops": {"t": t_stat, "p": t_p, "significant": t_p < 0.05, "cohens_d": effect_size}
    },
    "regression_model": {
        "r_squared": model.rsquared,
        "r_squared_adj": model.rsquared_adj,
        "aic": model.aic,
        "bic": model.bic
    },
    "clustering": {
        "optimal_clusters": optimal_k,
        "silhouette_score": max(silhouette_scores)
    },
    "market_segmentation": cluster_summary.to_dict(),
    "recommendations": [
        "Diferenças significativas entre airlines justificam estratégias diferenciadas",
        "Horários de partida influenciam preços - oportunidade de otimização",
        f"Voos diretos têm premium de ₹{abs(np.mean(direct_flights) - np.mean(flights_with_stops)):.0f}",
        f"Mercado naturalmente segmentado em {optimal_k} grupos distintos",
        f"Modelo preditivo explica {model.rsquared_adj*100:.1f}% da variação nos preços"
    ]
}

# Salvar resultados
import json
os.makedirs('../data/processed', exist_ok=True)

with open('../data/processed/statistical_analysis_results.json', 'w') as f:
    # Converter numpy types para tipos serializáveis
    def convert_numpy(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return obj
    
    json.dump(statistical_results, f, indent=2, default=convert_numpy)

# Salvar dataset com clusters
df_clustered.to_csv('../data/processed/flights_with_clusters.csv', index=False)

print(f"\n💾 RESULTADOS SALVOS:")
print(f"   📊 Análise estatística: ../data/processed/statistical_analysis_results.json")
print(f"   🎯 Dataset com clusters: ../data/processed/flights_with_clusters.csv")

print("\n" + "="*60)
print("🎯 ANÁLISE ESTATÍSTICA CONCLUÍDA COM SUCESSO!")
print("="*60)
print("\n🚀 PRÓXIMOS PASSOS:")
print("   1. Desenvolver modelos de machine learning")
print("   2. Criar dashboard interativo em Streamlit")
print("   3. Implementar sistema de recomendações")
print("   4. Validar insights com dados externos")